In [1]:
import os
import tensorflow as tf
from keras import layers
from keras import Model
from os import getcwd
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
from keras.optimizers import RMSprop
import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout,GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

C:\Users\joshi\Anaconda3\envs\tfcpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\joshi\Anaconda3\envs\tfcpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\joshi\Anaconda3\envs\tfcpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\joshi\Anaconda3\envs\tfcpu\lib\site-packages\tensorflow\python\f

In [2]:
model = keras.applications.VGG16(input_shape = (224, 224, 3), 
                                include_top = True, 
                                weights = 'imagenet')


for layer in model.layers:
    layer.trainable = False
    
model.summary()


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0    

In [3]:
last_layer = model.get_layer('fc2')
last_output = last_layer.output

x = keras.layers.Dense(2,activation='softmax')(last_output)           

model = Model( model.input, x) 

model.compile(optimizer = Adam(lr=0.00001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [4]:
data_dir = "C:\\Users\\joshi\\OneDrive\\Desktop\\NIT Surat\\COVID-CT-master\\" # Your Dataset Location Path
train_dir = os.path.join(data_dir, 'Train')
val_dir = os.path.join(data_dir, 'Val')
train_malaria_dir = os.path.join(train_dir, 'COVID') #Replace the folder names
train_uninfected_dir = os.path.join(train_dir, 'NON-COVID')
validation_malaria_dir = os.path.join(val_dir, 'COVID')
validation_uninfected_dir = os.path.join(val_dir, 'NON-COVID')

train_malaria_fnames = os.listdir(train_malaria_dir)
train_uninfected_fnames = os.listdir(train_uninfected_dir)
validation_malaria_fnames = os.listdir(validation_malaria_dir)
validation_uninfected_fnames = os.listdir(validation_uninfected_dir)

print(len(train_malaria_fnames))
print(len(validation_malaria_fnames))
print(len(train_uninfected_fnames))
print(len(validation_uninfected_fnames))

191
60
234
58


In [5]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   
                                   )


train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=8,
                                                    class_mode='categorical',
                                                    target_size=(224, 224)
                                                    )
validation_datagen = ImageDataGenerator(rescale=1./255)
 

validation_generator = validation_datagen.flow_from_directory(val_dir,
                                                              batch_size=8,
                                                              class_mode='categorical',
                                                              target_size=(224, 224))

Found 425 images belonging to 2 classes.
Found 118 images belonging to 2 classes.


In [6]:
history = model.fit_generator(train_generator,
            validation_data = validation_generator,
            epochs = 100)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/100
54/54 [==============================] - 133s 2s/step - loss: 0.8061 - acc: 0.4753 - val_loss: 0.6759 - val_acc: 0.5593
Epoch 2/100
54/54 [==============================] - 132s 2s/step - loss: 0.6851 - acc: 0.5647 - val_loss: 0.7616 - val_acc: 0.5339
Epoch 3/100
54/54 [==============================] - 135s 2s/step - loss: 0.6762 - acc: 0.5553 - val_loss: 0.6454 - val_acc: 0.5593
Epoch 4/100
54/54 [==============================] - 135s 2s/step - loss: 0.6749 - acc: 0.5741 - val_loss: 0.7115 - val_acc: 0.5424
Epoch 5/100
54/54 [==============================] - 138s 3s/step - loss: 0.6684 - acc: 0.5624 - val_loss: 0.6803 - val_acc: 0.5424
Epoch 6/100
54/54 [==============================] - 138s 3s/step - loss: 0.6664 - acc: 0.5882 - val_loss: 0.6365 - val_acc: 0.5763
Epoch 7/100
54/54 [==============================] - 141s 3s/step - loss: 0.6772 - acc: 0.5765 - val_loss: 0.6154

In [7]:
test_dir = os.path.join(data_dir, 'Test')
test_malaria_dir = os.path.join(test_dir, 'COVID') #Replace the folder names
test_uninfected_dir = os.path.join(test_dir, 'NON-COVID')
test_malaria_fnames = os.listdir(test_malaria_dir)
test_uninfected_fnames = os.listdir(test_uninfected_dir)
print(len(test_malaria_fnames))
print(len(test_uninfected_fnames))

98
105


In [8]:
testGenerator = ImageDataGenerator().flow_from_directory(test_dir, 
                                                      batch_size=8,
                                                    class_mode='categorical',shuffle=False,target_size=(224,224)
                                                      )

Found 203 images belonging to 2 classes.


In [17]:
import numpy as np
from sklearn.metrics import confusion_matrix,classification_report
batch_size = 8
num_of_test_samples = 203
predictions = model.predict_generator(testGenerator)

y_pred = np.argmax(predictions, axis=1)

true_classes = testGenerator.classes

class_labels = list(testGenerator.class_indices.keys())   

print(class_labels)

print(confusion_matrix(testGenerator.classes, y_pred))

report = classification_report(true_classes, y_pred, target_names=class_labels)
print(report)

['COVID', 'NON-COVID']
[[38 60]
 [15 90]]
              precision    recall  f1-score   support

       COVID       0.72      0.39      0.50        98
   NON-COVID       0.60      0.86      0.71       105

    accuracy                           0.63       203
   macro avg       0.66      0.62      0.60       203
weighted avg       0.66      0.63      0.61       203



In [18]:
metrics.roc_auc_score(y_pred, testGenerator.classes)

0.6584905660377359

In [19]:
metrics.accuracy_score(testGenerator.classes, y_pred)

0.6305418719211823

In [14]:
import numpy as np
import sklearn.metrics as metrics
score, acc = model.evaluate(testGenerator.classes, testGenerator)
print('Confusion Matrix')
cm = metrics.confusion_matrix(testGenerator.classes,testGenerator )
plt.imshow(cm, cmap=plt.cm.Blues)
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.xticks([], [])
plt.yticks([], [])
plt.title('Confusion matrix ')
plt.colorbar()
plt.show()

print('Classification Report')
print(metrics.classification_report(testGenerator.classes, y_pred))

ValueError: Error when checking input: expected input_1 to have 4 dimensions, but got array with shape (203, 1)